## 如果有choice的数据接口既可以实现一次性运行更新（每日更新，后续接入分钟级别的数据）

In [ ]:
"""
update_data.py
================

该模块用于自动拉取ETF、期货以及各类利率和利差数据，并生成最新的合并数据与因子库。

功能说明：

1. **拉取行情数据**：通过 akshare 获取富国7-10年政金债 ETF（511520）的历史行情以及指定国债期货合约的日行情。
2. **拉取利率曲线数据**：包括中债公布的1年、5年、10年、30年期国债到期收益率，10年期政策性金融债收益率，以及资金面指标（Shibor 3M、R007）和利率互换固定端（FR007-1Y）。如果您使用 Wind/Choice 数据源，可在相应函数中替换为调用 Wind/Choice 接口。
3. **数据合并与清洗**：将ETF、期货行情与利率数据按日期合并，统一日期格式；生成基础合并表格 ``etf_futures_interest_merged.csv``。
4. **因子计算**：调用 utils.factors_engine.add_factors() 计算因子，并输出 ``etf_futures_interest_factors.csv``。

"""

import pandas as pd
from datetime import datetime
import akshare as ak  # 如果本地环境未安装 akshare，请先安装或替换为 Wind/Choice 接口
from pathlib import Path
import sys

# 设置保存路径（本地绝对路径）
ETF_PATH = r"C:\Users\l\OneDrive\桌面\东方财富\511520  hedging strategy\data\etf_511520.csv"
FUTURES_PATH = r"C:\Users\l\OneDrive\桌面\东方财富\511520  hedging strategy\data\futures_T_specific.csv"

def fetch_etf_data():
    print("拉取富国7-10年政金债ETF（511520）历史行情...")
    df_etf = ak.fund_etf_hist_em(symbol="511520")
    df_etf.columns = [
        "date", "open", "close", "high", "low",
        "volume", "turnover", "amplitude", "pct_change", "chg", "turnover_rate"
    ]
    df_etf.to_csv(ETF_PATH, index=False, encoding='utf-8-sig')
    print(f"ETF数据保存成功：{ETF_PATH}")


#这里有一定的局限性，就是需要关注是否在当下有相应的国债期货数据供我，选择进行对冲

def fetch_futures_data():
    symbol = "T2509"
    df_fut = ak.futures_zh_daily_sina(symbol=symbol)
    print("期货数据字段：", df_fut.columns.tolist())
    # 只保留需要的字段
    df_fut = df_fut[["date", "open", "high", "low", "close", "volume", "hold"]]
    # 重命名字段
    df_fut.columns = ["date", "open", "high", "low", "close", "volume", "open_interest"]
    df_fut.to_csv(FUTURES_PATH, index=False, encoding='utf-8-sig')
    print(f"{symbol}合约期货数据保存成功：{FUTURES_PATH}")


In [ ]:
def fetch_interest_data(start_date: str = None, end_date: str = None) -> pd.DataFrame:
    """
    获取利率曲线、政策性金融债、资金面和利率互换数据。
    返回：包含日期及多个利率指标的 DataFrame。
    """
    if ak is None:
        raise RuntimeError("akshare 未安装，无法拉取利率数据。请在本地安装 akshare 或替换数据源。")

    print("Fetching yield curve data…")
    # 国债收益率（1年、5年、10年、30年）
    yield_curve = ak.bond_china_yield_zh()
    yield_curve["date"] = pd.to_datetime(yield_curve["日期"])
    yield_1y = yield_curve[["date", "1年"]].rename(columns={"1年": "中国:国债收益率:1年"})
    yield_5y = yield_curve[["date", "5年"]].rename(columns={"5年": "中国:国债收益率:5年"})
    yield_10y = yield_curve[["date", "10年"]].rename(columns={"10年": "中国:国债收益率:10年"})
    yield_30y = yield_curve[["date", "30年"]].rename(columns={"30年": "中国:国债收益率:30年"})

    # 政策性金融债收益率（10年）
    # 该接口可能需要查找具体函数，示例用 bond_zh_us_rate（如无请补充实际接口）
    try:
        policy_10y = ak.bond_zh_us_rate()
        policy_10y = policy_10y[policy_10y["期限"] == "10年"]
        policy_10y = policy_10y[["日期", "收盘收益率"]].rename(columns={"日期": "date", "收盘收益率": "中债国开债到期收益率:10年"})
        policy_10y["date"] = pd.to_datetime(policy_10y["date"])
    except Exception:
        policy_10y = None

    # Shibor 3M
    shibor = ak.macro_china_shibor_all()
    shibor["date"] = pd.to_datetime(shibor["日期"])
    shibor_3m = shibor[["date", "3个月"]].rename(columns={"3个月": "SHIBOR:3个月"})

    # R007（回购利率）
    r007 = ak.repo_rate_hist()
    r007["date"] = pd.to_datetime(r007["日期"])
    r007 = r007[["date", "R007"]].rename(columns={"R007": "R007"})

    # FR007利率互换（1年）
    try:
        fr007 = ak.macro_china_swap_rate()
        fr007["date"] = pd.to_datetime(fr007["日期"])
        irs_fr007 = fr007[fr007["期限"] == "1年"][["date", "FR007"]].rename(columns={"FR007": "利率互换:FR007:1年"})
    except Exception:
        irs_fr007 = None

    # 合并所有数据
    data_frames = [yield_1y, yield_5y, yield_10y, yield_30y, shibor_3m, r007]
    if policy_10y is not None:
        data_frames.append(policy_10y)
    if irs_fr007 is not None:
        data_frames.append(irs_fr007)

    from functools import reduce
    df_rates = reduce(lambda left, right: pd.merge(left, right, on="date", how="outer"), data_frames)
    df_rates = df_rates.sort_values("date").reset_index(drop=True)
    return df_rates


In [ ]:
def merge_and_save(etf_df: pd.DataFrame, fut_df: pd.DataFrame, rates_df: pd.DataFrame) -> None:
    """将ETF、期货和利率数据按日期合并并保存。

    文件保存至 DATA_DIR 目录下的 etf_futures_interest_merged.csv。
    """
    print("Merging data…")
    # 合并ETF和期货行情
    merged = pd.merge(etf_df, fut_df, on="date", how="inner")
    # 合并利率数据，使用外连接以保留行情日期
    merged = pd.merge(merged, rates_df, on="date", how="left")
    # 按日期排序
    merged = merged.sort_values("date").reset_index(drop=True)
    out_path = DATA_DIR / "etf_futures_interest_merged.csv"
    merged.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"合并行情与利率数据已保存至 {out_path}")
    return merged


def calculate_and_save_factors(merged_df: pd.DataFrame) -> pd.DataFrame:
    """对合并数据计算因子，并保存为 CSV。"""
    print("Calculating factors…")
    factor_df = add_factors(merged_df.copy())
    out_path = DATA_DIR / "etf_futures_interest_factors.csv"
    factor_df.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"含因子数据已保存至 {out_path}")
    return factor_df


def main(etf_symbol: str = "511520", futures_contract: str = "T2509"):
    """主函数：拉取数据、合并并计算因子。"""
    # 拉取行情
    etf_df = fetch_etf_data(etf_symbol)
    fut_df = fetch_futures_data(futures_contract)
    # 拉取利率数据
    rates_df = fetch_interest_data()
    # 合并并保存
    merged_df = merge_and_save(etf_df, fut_df, rates_df)
    # 计算因子并保存
    calculate_and_save_factors(merged_df)


if __name__ == "__main__":
    # 默认使用当前合约
    main()